In [1]:
import webdataset as wds

import sys

sys.path.append('..')
from dataloading import data_loading as dl

TypeError: unsupported operand type(s) for |: '_GenericAlias' and '_GenericAlias'

In [ ]:
def make_webDataloader(cfg, mode='train'):
    
    assert mode in ['train', 'val', 'test'], mode

    shard_list = dl.ShardList(
        cfg[f'urls_{mode}'],
        base_path=cfg['data_dir'],
        world_size=cfg['world_size'],
        rank=cfg['global_rank'],
        substitute_param=(mode == 'train'),
    )

    dataset = wds.WebDataset(shard_list)
    dataset.pipeline.pop()
    if mode == 'train': dataset.append(wds.shuffle(10))          # Shuffle order of shards
    dataset.append(wds.tariterators.tarfile_to_samples())        # Gather files inside tar files into samples
    dataset.append(wds.split_by_worker)                          # Use a different subset of samples in shards in different workers
    if mode == 'train': dataset.append(wds.shuffle(100))         # Shuffle samples within a worker process
    dataset.append(wds.decode('pill', dl.decode_xyz))            # Decode image and xyz files
    dataset.append(dl.rotate_and_stack(reverse=False))           # Combine separate images into a stack, reverse=True only for QUAM dataset
    dataset.append(dl.batched(cfg['batch_size']))                # Gather samples into batches
    #dataset = dataset.map(partial(apply_preprocessing, cfg=cfg)) # Preprocess batch

    dataloader = wds.WebLoader(
        dataset,
        num_workers=cfg['num_workers'],
        batch_size=None, # Batching is done in the WebDataset
        pin_memory=True,
        collate_fn=dl.default_collate,
        persistent_workers=False
    )
    
    return dataset, dataloader
